In [2]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-18])
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\')
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\')

In [3]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import f1_score, precision_score

from anchor import anchor_tabular

segment = Dataset()

In [4]:
X, y, feature_names, class_names = segment.load_segment()

In [7]:
y = [int(i) for i in y]
y = np.array(y)
one_h_percent = int(min(10*len(X)/100,100))
print("Instances:",one_h_percent)

Instances: 100


In [14]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [False],
        'min_samples_leaf' : [1],
        'n_estimators': [500]
    }]
    lf = LionForests(None, False, None, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task):

        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            rule = mdl.find_rule_length(instances[0])
            return rule, cover, def_predictions

        #Anchors =======================================================================================
        explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0

        #CHIRPS =======================================================================================
        project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
        temp_y = np.array([int(i+1) for i in y_train])
        temp_frame = pd.DataFrame(np.hstack((train,temp_y.reshape(len(temp_y),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)
        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'segment', random_state=123)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)

        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=123,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance)[5]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification')
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        covered = False
                    if instance[k] <= rule[feature][1][1]:
                        covered = False
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        covered = False
                else:
                     if instance[k] > rule[feature][0][1]: #<=
                        covered = False
        if covered:
            return 1
        else:
            return 0

    gt = GlobalSurrogateTree(train,lf.model.predict(train),feature_names,'c')
    lt = LocalSurrogateTree(train,lf.model.predict(train),feature_names,'c',150)

    loo = LeaveOneOut()
    loo.get_n_splits(test)
    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    for train_index, test_index in loo.split(test):
        x_train_temp, x_test_temp = test[train_index], test[test_index]
        y_train_temp, y_test_temp = test_predictions[train_index], test_predictions[test_index]
        x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
        x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op = method(test_index[0])
                coverage = 0
                precision = []
                co = 0
                for i in x_train_temp_lf:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions = method(np.concatenate((x_test_temp,x_train_temp)))
                f_precision[name].append(precision_score(predictions,y_train_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
            elif name == 'lf':
                rule = method(x_test_temp_lf[0])
                coverage = 0
                precision = []
                co = 0
                for i in x_train_temp_lf:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            else:
                rule, prediction = method(x_test_temp[0])
                coverage = 0
                precision = []
                co = 0 
                for i in x_train_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_train_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision

In [15]:
#100 instances
total_results = []
rs = [7, 77, 777, 7777, 77777, 5, 55, 555, 5555, 55555]
for random_s in rs:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=one_h_percent, stratify=y, random_state=random_s)
    results = measure(X_train, X_test, y_train, y_test, feature_names, class_names)
    total_results.append(results)

len self.path_detail
100
Working on CHIRPS for instance 0 of 100
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
reduced 3 patterns out of 207 by numeric redundancy
found 204 patterns from 500 trees for batch_idx 0
start score sort for batch_idx 0 (204) patterns
start merge rule for batch_idx 0 (204) patterns
[('hue-mean', False, 0.14556), ('rawred-mean', False, 0.18112), ('region-centroid-row', True, 0.61875), ('intensity-mean', True, 0.57849), ('rawgreen-mean', False, 0.16017)]
0.9742647058823529 0.11947700631199279 0.30461089457264323 0.35357053993530735
merge complete for batch_idx 0 (204) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
reduced 55 patterns out of 914 by numeric redundancy
found 859 patterns from 293 trees for batch_idx 1
start score sort for batch_idx 1 (859) patterns
start merge rule for batch_idx 1 (859) patterns
[('hue-mean', True, 0.19602), ('hue-mean', False, 0.138

reduced 35 patterns out of 904 by numeric redundancy
found 869 patterns from 500 trees for batch_idx 14
start score sort for batch_idx 14 (869) patterns
start merge rule for batch_idx 14 (869) patterns
[('exgreen-mean', True, 0.46015), ('hue-mean', False, 0.1988), ('vedge-mean', True, 0.09601), ('rawred-mean', True, 0.18112), ('exred-mean', False, 0.65578), ('rawblue-mean', True, 0.67452), ('region-centroid-row', True, 0.60621)]
0.975177304964539 0.12398557258791704 0.09862132688571329 0.1215000213739156
merge complete for batch_idx 14 (869) patterns
start get explainer for batch_idx 14
Working on CHIRPS for instance 15 of 100
as_chirps for batch_idx 15
start mining for batch_idx 15 with support = 0.1
found 3 patterns from 500 trees for batch_idx 15
start score sort for batch_idx 15 (3) patterns
start merge rule for batch_idx 15 (3) patterns
[('rawblue-mean', False, 0.5495213568210602)]
0.9294117647058824 0.1487883971630531 0.4445948145460821 0.33333333333333337
merge complete for batc

reduced 62 patterns out of 665 by numeric redundancy
found 603 patterns from 330 trees for batch_idx 29
start score sort for batch_idx 29 (603) patterns
start merge rule for batch_idx 29 (603) patterns
[('hue-mean', False, 0.13882), ('rawred-mean', True, 0.01538), ('region-centroid-row', False, 0.5021), ('region-centroid-col', False, 0.49035)]
0.8831168831168831 0.03152654946462864 0.08484772184363673 0.13032440889098765
merge complete for batch_idx 29 (603) patterns
start get explainer for batch_idx 29
Working on CHIRPS for instance 30 of 100
as_chirps for batch_idx 30
start mining for batch_idx 30 with support = 0.1
found 35 patterns from 500 trees for batch_idx 30
start score sort for batch_idx 30 (35) patterns
start merge rule for batch_idx 30 (35) patterns
[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.61767), ('intensity-mean', False, 0.16183)]
0.9781931464174455 0.14156898106402163 0.23770253899342203 0.37650260830120197
merge complete for batch_idx 30 (35) patte

[('hue-mean', False, 0.5287514174124226)]
0.9781931464174455 0.14156898106402163 0.23861981415716968 0.27790262172284647
merge complete for batch_idx 45 (8) patterns
start get explainer for batch_idx 45
as_chirps for batch_idx 46
start mining for batch_idx 46 with support = 0.1
reduced 1 patterns out of 299 by numeric redundancy
found 298 patterns from 500 trees for batch_idx 46
start score sort for batch_idx 46 (298) patterns
start merge rule for batch_idx 46 (298) patterns
[('exgreen-mean', False, 0.48821), ('hue-mean', False, 0.18878), ('hue-mean', True, 0.61909), ('region-centroid-col', False, 0.45181)]
0.8823529411764706 0.027487695122585743 0.06720584608294879 0.11395650762739375
merge complete for batch_idx 46 (298) patterns
start get explainer for batch_idx 46
as_chirps for batch_idx 47
start mining for batch_idx 47 with support = 0.1
reduced 3 patterns out of 237 by numeric redundancy
found 234 patterns from 500 trees for batch_idx 47
start score sort for batch_idx 47 (234) pa

[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.61767), ('intensity-mean', False, 0.16183)]
0.9781931464174455 0.14156898106402163 0.23770262984981608 0.37650260830120197
merge complete for batch_idx 60 (35) patterns
start get explainer for batch_idx 60
as_chirps for batch_idx 61
start mining for batch_idx 61 with support = 0.1
found 3 patterns from 500 trees for batch_idx 61
start score sort for batch_idx 61 (3) patterns
start merge rule for batch_idx 61 (3) patterns
[('rawblue-mean', False, 0.5495213568210602)]
0.9294117647058824 0.1487883971630531 0.4445941530949022 0.33333333333333337
merge complete for batch_idx 61 (3) patterns
start get explainer for batch_idx 61
as_chirps for batch_idx 62
start mining for batch_idx 62 with support = 0.1
found 3 patterns from 500 trees for batch_idx 62
start score sort for batch_idx 62 (3) patterns
start merge rule for batch_idx 62 (3) patterns
[('rawblue-mean', False, 0.5495213568210602)]
0.9294117647058824 0.1487883971630531 0.44

...

start merge rule for batch_idx 28 (8) patterns
[('hue-mean', False, 0.46047999430447817)]
0.8532608695652174 0.158720338442407 0.23503661993667152 0.27802690582959644
merge complete for batch_idx 28 (8) patterns
start get explainer for batch_idx 28
as_chirps for batch_idx 29
start mining for batch_idx 29 with support = 0.1
reduced 4 patterns out of 211 by numeric redundancy
found 207 patterns from 500 trees for batch_idx 29
start score sort for batch_idx 29 (207) patterns
start merge rule for batch_idx 29 (207) patterns
[('hue-mean', False, 0.1458), ('rawred-mean', False, 0.18112), ('region-centroid-row', True, 0.61875), ('intensity-mean', True, 0.57849), ('saturatoin-mean', True, 0.40251)]
0.9742647058823529 0.11947700631199279 0.2237591018549194 0.2762541479601798
merge complete for batch_idx 29 (207) patterns
start get explainer for batch_idx 29
Working on CHIRPS for instance 30 of 100
as_chirps for batch_idx 30
start mining for batch_idx 30 with support = 0.1
reduced 4 patterns out

[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.61458), ('intensity-mean', False, 0.16189)]
0.9723926380368099 0.1436713916949067 0.2428621290818569 0.38247863247863245
merge complete for batch_idx 44 (33) patterns
start get explainer for batch_idx 44
Working on CHIRPS for instance 45 of 100
as_chirps for batch_idx 45
start mining for batch_idx 45 with support = 0.1
reduced 30 patterns out of 185 by numeric redundancy
found 155 patterns from 478 trees for batch_idx 45
start score sort for batch_idx 45 (155) patterns
start merge rule for batch_idx 45 (155) patterns
[('hue-mean', True, 0.16772), ('rawred-mean', True, 0.17329), ('region-centroid-row', True, 0.61844), ('region-centroid-row', False, 0.21595), ('saturatoin-mean', False, 0.48735), ('exgreen-mean', False, 0.47528), ('hedge-sd', False, 0.0001)]
0.8451882845188284 0.10294630888735992 0.40114068102132616 0.4819829777826909
merge complete for batch_idx 45 (155) patterns
start get explainer for batch_idx 45
as_chirps

[('hue-mean', False, 0.46047999430447817)]
0.8532608695652174 0.158720338442407 0.23503661993667152 0.27802690582959644
merge complete for batch_idx 59 (8) patterns
start get explainer for batch_idx 59
Working on CHIRPS for instance 60 of 100
as_chirps for batch_idx 60
start mining for batch_idx 60 with support = 0.1
reduced 103 patterns out of 414 by numeric redundancy
found 311 patterns from 487 trees for batch_idx 60
start score sort for batch_idx 60 (311) patterns
start merge rule for batch_idx 60 (311) patterns
[('hue-mean', True, 0.18098), ('rawred-mean', True, 0.1809), ('region-centroid-row', True, 0.50328), ('saturatoin-mean', False, 0.70919)]
0.90625 0.08292533064006447 0.1858834642972912 0.2829898886471266
merge complete for batch_idx 60 (311) patterns
start get explainer for batch_idx 60
as_chirps for batch_idx 61
start mining for batch_idx 61 with support = 0.1
found 3 patterns from 500 trees for batch_idx 61
start score sort for batch_idx 61 (3) patterns
start merge rule f

reduced 75 patterns out of 705 by numeric redundancy
found 630 patterns from 498 trees for batch_idx 75
start score sort for batch_idx 75 (630) patterns
start merge rule for batch_idx 75 (630) patterns
[('hue-mean', True, 0.19967), ('hue-mean', False, 0.13794), ('rawred-mean', True, 0.18032), ('region-centroid-row', True, 0.46015), ('saturatoin-mean', True, 0.88951), ('exgreen-mean', True, 0.47093)]
0.9370629370629371 0.0612514095252317 0.23170805487138935 0.3154698694267039
merge complete for batch_idx 75 (630) patterns
start get explainer for batch_idx 75
as_chirps for batch_idx 76
start mining for batch_idx 76 with support = 0.1
found 3 patterns from 500 trees for batch_idx 76
start score sort for batch_idx 76 (3) patterns
start merge rule for batch_idx 76 (3) patterns
[('rawblue-mean', False, 0.5495213568210602)]
0.9349112426035503 0.14805227557689427 0.44280864012816606 0.32727272727272727
merge complete for batch_idx 76 (3) patterns
start get explainer for batch_idx 76
as_chirps 

start merge rule for batch_idx 89 (706) patterns
[('hue-mean', True, 0.20128), ('hue-mean', False, 0.15846), ('rawred-mean', True, 0.04087), ('rawred-mean', False, 0.0019), ('region-centroid-row', True, 0.61457), ('saturatoin-mean', True, 0.88954), ('intensity-mean', True, 0.57849)]
0.9036144578313253 0.03424691627467985 0.2932868478748016 0.3764221632824622
merge complete for batch_idx 89 (706) patterns
start get explainer for batch_idx 89
Working on CHIRPS for instance 90 of 100
as_chirps for batch_idx 90
start mining for batch_idx 90 with support = 0.1
reduced 81 patterns out of 704 by numeric redundancy
found 623 patterns from 500 trees for batch_idx 90
start score sort for batch_idx 90 (623) patterns
start merge rule for batch_idx 90 (623) patterns
[('hue-mean', True, 0.19885), ('hue-mean', False, 0.1388), ('rawred-mean', True, 0.17971), ('region-centroid-row', True, 0.45498), ('saturatoin-mean', True, 0.88816), ('exgreen-mean', True, 0.47104)]
0.9366197183098591 0.060801031514016

In [16]:
#Me '0.75'
full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/one_h_percent)
        rule_length[name].append(i[2][name]/one_h_percent)
        f_precision[name].append(np.array(i[3][name]).mean())
for name, method in i[0].items():
    print(name,np.array(full_coverage[name]).mean(),',',np.array(full_coverage[name]).std(),
          np.array(rule_length[name]).mean(),',',np.array(rule_length[name]).std(),
          np.array(f_precision[name]).mean(),',',np.array(f_precision[name]).std())

gs 0.09282828282828283 , 0.006199914096386451 3.729 , 0.4088875150943105 0.9860838623403373 , 0.017836697542138076
ls 0.4437373737373739 , 0.014998010275134432 1.589 , 0.09771898484941399 0.3758100683772073 , 0.023161896099916396
an 0.0561111111111111 , 0.003669114579259564 4.886 , 0.19880643852752855 0.12840889109653503 , 0.024244090213287786
lf 0.07048484848484848 , 0.004890140919902875 8.764 , 0.4020994901762498 1.0 , 0.0
df 0.9869999999999998 , 0.013453624047074004 10.097999999999999 , 1.631685018623386 0.67 , 0.030331501776206277
ch 0.28078787878787875 , 0.008148930034556117 3.4490000000000003 , 0.1506286825275983 0.4310500112078371 , 0.00948842497186877
